## There are some instructions you need to follow:
<li> You only need to write your code in the comment area "Your Code Here".</li>
<li>Do not upload your own file. Please make the necessary changes in the Jupyter notebook file already present in the server.</li>
<li>Please note, there are several cells in the Assignment Jupyter notebook that are empty and read only. Do not attempt to remove them or   edit them. They are used in grading your notebook. Doing so might lead to 0 points.</li>

In [2]:
import nltk
import os
import _sqlite3
from nltk.corpus import PlaintextCorpusReader
from nltk import sent_tokenize,word_tokenize 
from gensim import corpora, models, similarities
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.similarities.docsim import Similarity

# Question 1

In [160]:
"""
Question 1

Write a function that takes the file name of the Wikipedia page containing all Greek ancient
philosophers (saved as "Index.html" in your workspace) and returns a list tuples containing 
the name of the philosopher and the path to its individual article file.

Example of use: get_philosophers("Index.html")

The output should be a list of tuples:

[('Acrion', 'Philosophers/Acrion.html'),
 ('Adrastus of Aphrodisias', 'Philosophers/Adrastus of Aphrodisias.html'),
 ('Aedesia', 'Philosophers/Aedesia.html'),
 ('Aedesius', 'Philosophers/Aedesius.html'),
 ('Aeneas of Gaza', 'Philosophers/Aeneas of Gaza.html'),
 ('Aenesidemus', 'Philosophers/Aenesidemus.html'),
 ...]
 
  
NOTE: For processing speed purposes, the table in "Index.html" has been shortened compared
to the one online on wikipedia.org. Do not worry if you do not find some philosophers in 
your results, this is made on purpose. 

"""

def get_philosophers(filename):
    
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(filename, 'r', 'utf-8')
    soup = BeautifulSoup(f.read(),'lxml')
    
    philosopher_tuple_list = []
    
    tableRows = soup.find_all('table',class_='wikitable sortable')[0]
    data_rows = tableRows.find_all('tr')[1:]
    philosopher_name = [] #list containing each philosopher's name
    # grabs philosopher name from title an appends to list
    for data in data_rows:
        philosopher_name.append(data.find('a').get('title'))

    # adding path and filename at the end of philosopher's name
    for name in philosopher_name:
        x = name
        y = name,"Philosophers/"+ x + '.html'
        return philosopher_tuple_list.append(y)

# Once done, try this:
print(get_philosophers("Index.html"))

None


In [12]:
def get_philosophers(filename):
    
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(filename, 'r', 'utf-8')
    soup = BeautifulSoup(f.read(),'lxml')
    
    philosopher_list = []
    
    tableRows = soup.find_all('table',class_='wikitable sortable')[0]
    data_rows = tableRows.find_all('tr')[1:]
    philosopher_name = [] #list containing each philosopher's name
    # grabs philosopher name from title an appends to list
    for data in data_rows:
        philosopher_name.append(data.find('a').get('title'))

    # adding path and filename at the end of philosopher's name
    for name in philosopher_name:
        x = name
        y = name,"Philosophers/"+ x + '.html'
        philosopher_list.append(y)

#   returns philosopher list as tuple
#     philosopher_tuple_list = tuple(philosopher_list)
    return philosopher_list

# Once done, try this:
get_philosophers("Index.html")

[('Acrion', 'Philosophers/Acrion.html'),
 ('Adrastus of Aphrodisias', 'Philosophers/Adrastus of Aphrodisias.html'),
 ('Aedesia', 'Philosophers/Aedesia.html'),
 ('Aedesius', 'Philosophers/Aedesius.html'),
 ('Aeneas of Gaza', 'Philosophers/Aeneas of Gaza.html'),
 ('Aenesidemus', 'Philosophers/Aenesidemus.html'),
 ('Aesara', 'Philosophers/Aesara.html'),
 ('Aeschines of Neapolis', 'Philosophers/Aeschines of Neapolis.html'),
 ('Aeschines of Sphettus', 'Philosophers/Aeschines of Sphettus.html'),
 ('Aetius of Antioch', 'Philosophers/Aetius of Antioch.html'),
 ('Agapius (philosopher)', 'Philosophers/Agapius (philosopher).html'),
 ('Agathobulus', 'Philosophers/Agathobulus.html'),
 ('Agathosthenes', 'Philosophers/Agathosthenes.html'),
 ('Agrippa the Skeptic', 'Philosophers/Agrippa the Skeptic.html'),
 ('Albinus (philosopher)', 'Philosophers/Albinus (philosopher).html'),
 ('Alcinous (philosopher)', 'Philosophers/Alcinous (philosopher).html'),
 ('Alcmaeon of Croton', 'Philosophers/Alcmaeon of Crot

# Question 1: Non-Function

In [13]:
import codecs
from bs4 import BeautifulSoup
f = codecs.open("Index.html", 'r', 'utf-8')
soup = BeautifulSoup(f.read(),'lxml')

# reference: http://savvastjortjoglou.com/nba-draft-part01-scraping.html
# tuples of philosopher name w/ path 
philosopher_list = []

# creates list of philosopher name
tableRows = soup.find_all('table',class_='wikitable sortable')[0]
data_rows = tableRows.find_all('tr')[1:]
philosopher_name = [] #list containing each philosopher's name

# grabs philosopher name from title and appends to list
for data in data_rows:
    philosopher_name.append(data.find('a').get('title'))


# # loops through a list indices 
# for i in range(len(data_rows)):
#     # for each table data element from each table row
#     td = data_rows[i].find_all('td')[0].get_text().strip()
# #     philosopher_name.append(td)


# adding path and filename at the end of philosopher's name
for name in philosopher_name:
    x = name
    y = name,"Philosophers/"+ x + '.html'
    philosopher_list.append(y)

# philosopher_tuple_list = tuple(philosopher_list)
# philosopher_tuple_list

# Question 2

In [28]:
"""
Question 2


Write a function that scrapes the text on a philosophers’s page and returns it as a text 
string. The input is the name of the file that contains the philosoph's page.

Example of use: get_text('Philosophers/Acrion.html')
should output the text of the page.
'Acrion was a Locrian and a Pythagorean philosopher...'
"""

def get_text(file):
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(file, 'r', 'utf-8')
    page_soup = BeautifulSoup(f.read(),'lxml')

    all_text = ""

    for tag in page_soup.find_all('p'):
        all_text += tag.get_text()
    return all_text
# Once done, try this:
get_text("Philosophers/Acrion.html")

'Acrion was a Locrian and a Pythagorean philosopher.[1]  He is mentioned by Valerius Maximus[2] under the name of Arion. According to William Smith, Arion is a false reading, instead of Acrion.[3]\n'

# Question 2: Non-Function

In [23]:
# looks at page index.html (all philosophers)
filename = "Philosophers/Acrion.html"

import codecs
from bs4 import BeautifulSoup
f = codecs.open(filename, 'r', 'utf-8')
page_soup = BeautifulSoup(f.read(),'lxml')


all_text = ""

# takes the 2nd index from list philosopher_list
for philosopher_filepath in philosopher_list:
    philosopher_filepath = philosopher_filepath[1]
    
    filepath = codecs.open(philosopher_filepath, 'r', 'utf-8')
    page_soup = BeautifulSoup(filepath.read(),'lxml')
    
    for tag in page_soup.find_all('p'):
        all_text += tag.get_text()
# all_text

In [307]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [308]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [309]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
